In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [2]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['SHGData_20mW_07042017.txt', 'SHGData_10mW_07042017.txt', 'SHGData_0mW_07042017.txt', 'SHGData_30mWW_07042017.txt', 'SHGData_40mWW_07042017.txt', 'SHGData_50mWW_07042017.txt', 'SHGData_60mWW_07042017.txt', 'SHGData_70mWW_07042017.txt', 'SHGData_80mWW_07042017.txt', 'SHGData_90mWW_07042017.txt', 'SHGData_100mWW_07042017.txt', 'SHGData_110mWW_07042017.txt', 'SHGData_120mWW_07042017.txt']


In [3]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [4]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [5]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (499*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = ((x[i]*itr + offset)*25.4*2/(10**-12*c))-T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [21]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,300):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(300,len(allFiles))
PDvalues = PDvalues.values.reshape(300,len(allFiles)) 
timeDelay = ConvertToTimeDelay(times,itr,offset) # function converts to timedelay

MeanPD = PDvalues.mean(axis = 0) #Getting mean of all the data
StDevPD = PDvalues.std(axis = 0) # Getting Standard Deviation of the data
ErrOfMean = (StDevPD)/(300**0.5) # Standard error of mean stdev/sqrt(#ofdatapoints)
Background = MeanPD[2] # Background is the last element in the allFiles
ActualPD = np.absolute(MeanPD - Background)
RealPD = np.delete(ActualPD,2)
LogPD = np.log10(RealPD)
LaserPower = np.array([20,10,30,40,50,60,70,80,90,100,110,120])
LogLaser = np.log10(LaserPower)
print(RealPD)
error = np.delete(ErrOfMean,2)
print(error)
print(error/(log(10)*RealPD))
print(LogLaser)

[ 0.00254967  0.00136667  0.010186    0.02118167  0.02153333  0.03085167
  0.03832133  0.05460433  0.06251433  0.073599    0.084542    0.09519933]
[ 0.00047317  0.00025437  0.00018672  0.00017165  0.00024023  0.00021038
  0.00017569  0.00027438  0.00033672  0.00028555  0.00024627  0.00023371]
[ 0.08059646  0.08083419  0.00796104  0.0035194   0.00484515  0.00296151
  0.00199111  0.00218226  0.00233924  0.00168495  0.0012651   0.00106616]
[ 1.30103     1.          1.47712125  1.60205999  1.69897     1.77815125
  1.84509804  1.90308999  1.95424251  2.          2.04139269  2.07918125]


In [40]:
Delta = len(LogPD)*sum(LogLaser**2) - (sum(LogLaser))**2
A = (sum(LogLaser**2)*sum(LogPD) - sum(LogLaser)*sum(LogLaser*LogPD))/Delta
B = (len(LogPD)*sum(LogLaser*LogPD) - (sum(LogPD)*sum(LogLaser)))/Delta
# Least Squares Fitting Parameters above
print(A)
print(B)
# Errors in Least Squares Plotting
err = (((len(LogPD)-2)**-1) * sum((LogPD - A -B*LogLaser)**2)**0.5)
errorA = err*(sum(LogLaser**2)/Delta)**0.5
errorB = err*(len(LogPD)/Delta)**0.5
print(errorA)
print(errorB)

plt.plot(LogLaser, B*LogLaser + A, 'k')
plt.plot(LogLaser,LogPD,'b*',label = 'Data')
plt.show()
# fit = np.polyfit(LogLaser,LogPD,1,w=error/(log(10)*RealPD))
# f = np.poly1d(fit)
# pPower = np.linspace(LogLaser[1],LogLaser[-1],100)
# f_new = f(pPower)
# print(f_new)
# print(f)

-4.70482993481
1.79134820908
0.0468500407739
0.0267441887967


In [23]:
plt.plot(LogLaser,LogPD,'b*',label = 'Data')
plt.plot(pPower,f_new,'k-',label = 'Fit: y = 1.927x-4.98')
plt.plot()
plt.grid(True)
plt.xlabel('Log(Laser Power)')
plt.ylabel('Log(PMT Signal)')
plt.legend(loc = 'upper left')
plt.show()
#plt.savefig('THz_PMTSignal_07042017.png')

In [ ]:
print(len(RealPD))

In [16]:
log(10)

2.302585092994046